#  AI Fashion Assistant v2.4 - User Management System

**User Profile, Search History & Favorites Management**

---

**Project:** AI Fashion Assistant (TÜBİTAK 2209-A)  
**Student:** Hatice Baydemir  
**Date:** January 5, 2026  
**Version:** 2.4.0

---

## 📋 Features

- ✅ User profile management
- ✅ Search history tracking
- ✅ Favorites system
- ✅ Thread-safe JSON storage
- ✅ Analytics & statistics

---

## PART 1: Setup

In [1]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/ai_fashion_assistant_v2')

print('✅ Drive mounted')
print(f'📂 Working directory: {os.getcwd()}')

Mounted at /content/drive
✅ Drive mounted
📂 Working directory: /content/drive/MyDrive/ai_fashion_assistant_v2


In [2]:
# Clean start - delete old data if exists
import shutil
from pathlib import Path

data_dir = Path('v2.4-complete/data/users')

if data_dir.exists():
    shutil.rmtree(data_dir)
    print('🗑️  Old data deleted')

# Create fresh directories
data_dir.mkdir(parents=True, exist_ok=True)
Path('v2.4-complete/notebooks').mkdir(parents=True, exist_ok=True)
Path('v2.4-complete/src').mkdir(parents=True, exist_ok=True)

print('✅ Fresh directories created')
print(f'📁 Storage: {data_dir}')

🗑️  Old data deleted
✅ Fresh directories created
📁 Storage: v2.4-complete/data/users


---

## PART 2: Imports

In [3]:
import json
import logging
from datetime import datetime, timedelta
from typing import Dict, List, Optional, Tuple, Any
from dataclasses import dataclass, field, asdict
from pathlib import Path
from collections import Counter
from enum import Enum
import threading

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(levelname)s - %(message)s')
logger = logging.getLogger('v2.4')

logger.info('AI Fashion Assistant v2.4 - User Management')
print('✅ Imports loaded')

✅ Imports loaded


---

## PART 3: Enums

In [4]:
class StylePreference(Enum):
    CASUAL = "casual"
    FORMAL = "formal"
    SPORTY = "sporty"
    ELEGANT = "elegant"
    STREETWEAR = "streetwear"
    VINTAGE = "vintage"

class Size(Enum):
    XS = "xs"
    S = "s"
    M = "m"
    L = "l"
    XL = "xl"
    XXL = "xxl"

print('✅ Enums defined')
print(f'   Styles: {[s.value for s in StylePreference]}')
print(f'   Sizes: {[s.value for s in Size]}')

✅ Enums defined
   Styles: ['casual', 'formal', 'sporty', 'elegant', 'streetwear', 'vintage']
   Sizes: ['xs', 's', 'm', 'l', 'xl', 'xxl']


---

## PART 4: Data Models

In [5]:
@dataclass
class UserPreferences:
    """User preferences for fashion recommendations"""
    style: List[str] = field(default_factory=list)
    size: str = "M"
    colors: List[str] = field(default_factory=list)
    categories: List[str] = field(default_factory=list)

    def to_dict(self) -> Dict:
        return asdict(self)

    @classmethod
    def from_dict(cls, data: Dict):
        return cls(**data)

@dataclass
class UserProfile:
    """Complete user profile"""
    user_id: str
    name: str
    email: Optional[str] = None
    preferences: UserPreferences = field(default_factory=UserPreferences)
    created_at: str = field(default_factory=lambda: datetime.now().isoformat())
    last_active: str = field(default_factory=lambda: datetime.now().isoformat())
    total_searches: int = 0
    total_favorites: int = 0

    def __post_init__(self):
        if isinstance(self.preferences, dict):
            self.preferences = UserPreferences.from_dict(self.preferences)

    def to_dict(self) -> Dict:
        data = asdict(self)
        data['preferences'] = self.preferences.to_dict()
        return data

    @classmethod
    def from_dict(cls, data: Dict):
        return cls(**data)

    def update_activity(self):
        self.last_active = datetime.now().isoformat()

@dataclass
class SearchEntry:
    """Search history entry"""
    query: str
    timestamp: str
    results_count: int
    top_result_id: Optional[str] = None
    response_time: float = 0.0

    def to_dict(self) -> Dict:
        return asdict(self)

    @classmethod
    def from_dict(cls, data: Dict):
        return cls(**data)

@dataclass
class FavoriteProduct:
    """Favorite product"""
    product_id: str
    product_name: str
    category: str
    added_at: str
    view_count: int = 0
    last_viewed: Optional[str] = None

    def to_dict(self) -> Dict:
        return asdict(self)

    @classmethod
    def from_dict(cls, data: Dict):
        return cls(**data)

    def record_view(self):
        self.view_count += 1
        self.last_viewed = datetime.now().isoformat()

print('✅ Data models defined')

✅ Data models defined


---

## PART 5: Storage Layer

In [6]:
class ThreadSafeJSONStorage:
    """Thread-safe JSON storage"""

    def __init__(self, filepath: Path):
        self.filepath = Path(filepath)
        self._lock = threading.Lock()
        self.filepath.parent.mkdir(parents=True, exist_ok=True)

    def load(self) -> Dict:
        with self._lock:
            if not self.filepath.exists():
                return {}
            try:
                with open(self.filepath, 'r', encoding='utf-8') as f:
                    return json.load(f)
            except json.JSONDecodeError:
                logger.error(f"Corrupted: {self.filepath}")
                return {}

    def save(self, data: Dict):
        with self._lock:
            with open(self.filepath, 'w', encoding='utf-8') as f:
                json.dump(data, f, indent=2, ensure_ascii=False)

    def update(self, key: str, value: Any):
        data = self.load()
        data[key] = value
        self.save(data)

print('✅ Storage layer defined')

✅ Storage layer defined


---

## PART 6: Manager Classes

In [7]:
class UserProfileManager:
    """User profile management"""

    def __init__(self, storage_dir: Path):
        self.storage_dir = Path(storage_dir)
        self.storage = ThreadSafeJSONStorage(self.storage_dir / 'users.json')
        self._cache: Dict[str, UserProfile] = {}
        self._load_all()

    def _load_all(self):
        data = self.storage.load()
        for uid, udata in data.items():
            self._cache[uid] = UserProfile.from_dict(udata)
        logger.info(f"Loaded {len(self._cache)} users")

    def _save(self, user: UserProfile):
        self.storage.update(user.user_id, user.to_dict())
        self._cache[user.user_id] = user

    def create_user(self, user_id: str, name: str, email: str = None) -> UserProfile:
        if user_id in self._cache:
            raise ValueError(f"User {user_id} exists")
        user = UserProfile(user_id=user_id, name=name, email=email)
        self._save(user)
        logger.info(f"Created: {user_id}")
        return user

    def get_user(self, user_id: str) -> Optional[UserProfile]:
        return self._cache.get(user_id)

    def update_preferences(self, user_id: str, **kwargs) -> UserProfile:
        user = self.get_user(user_id)
        if not user:
            raise ValueError(f"User {user_id} not found")
        for key, value in kwargs.items():
            if hasattr(user.preferences, key):
                setattr(user.preferences, key, value)
        user.update_activity()
        self._save(user)
        return user

    def list_users(self) -> List[str]:
        return list(self._cache.keys())

    def get_stats(self) -> Dict:
        users = list(self._cache.values())
        if not users:
            return {'total_users': 0}
        return {
            'total_users': len(users),
            'avg_searches': sum(u.total_searches for u in users) / len(users),
            'avg_favorites': sum(u.total_favorites for u in users) / len(users),
        }

print('✅ UserProfileManager defined')

✅ UserProfileManager defined


In [8]:
class SearchHistoryManager:
    """Search history management"""

    def __init__(self, user_id: str, storage_dir: Path):
        self.user_id = user_id
        self.storage = ThreadSafeJSONStorage(storage_dir / f'history_{user_id}.json')
        self.history: List[SearchEntry] = self._load()

    def _load(self) -> List[SearchEntry]:
        data = self.storage.load()
        if isinstance(data, list):
            return [SearchEntry.from_dict(e) for e in data]
        return []

    def _save(self):
        data = [e.to_dict() for e in self.history]
        self.storage.save(data)

    def add_search(self, query: str, results_count: int, top_result_id: str = None,
                   response_time: float = 0.0) -> SearchEntry:
        entry = SearchEntry(
            query=query,
            timestamp=datetime.now().isoformat(),
            results_count=results_count,
            top_result_id=top_result_id,
            response_time=response_time
        )
        self.history.append(entry)
        self._save()
        return entry

    def get_recent(self, n: int = 10) -> List[SearchEntry]:
        return self.history[-n:]

    def get_patterns(self) -> Dict:
        if not self.history:
            return {'total': 0}
        queries = [e.query.lower() for e in self.history]
        words = []
        for q in queries:
            words.extend(q.split())
        return {
            'total': len(self.history),
            'unique': len(set(queries)),
            'top_terms': dict(Counter(words).most_common(5)),
            'avg_response': sum(e.response_time for e in self.history) / len(self.history)
        }

print('✅ SearchHistoryManager defined')

✅ SearchHistoryManager defined


In [9]:
class FavoritesManager:
    """Favorites management"""

    def __init__(self, user_id: str, storage_dir: Path):
        self.user_id = user_id
        self.storage = ThreadSafeJSONStorage(storage_dir / f'favorites_{user_id}.json')
        self.favorites: Dict[str, FavoriteProduct] = self._load()

    def _load(self) -> Dict[str, FavoriteProduct]:
        data = self.storage.load()
        return {pid: FavoriteProduct.from_dict(pdata) for pid, pdata in data.items()}

    def _save(self):
        data = {pid: prod.to_dict() for pid, prod in self.favorites.items()}
        self.storage.save(data)

    def add(self, product_id: str, product_name: str, category: str) -> Tuple[bool, str]:
        if product_id in self.favorites:
            return False, "Already in favorites"
        fav = FavoriteProduct(
            product_id=product_id,
            product_name=product_name,
            category=category,
            added_at=datetime.now().isoformat()
        )
        self.favorites[product_id] = fav
        self._save()
        return True, "Added"

    def record_view(self, product_id: str):
        if product_id in self.favorites:
            self.favorites[product_id].record_view()
            self._save()

    def get_all(self) -> List[FavoriteProduct]:
        return list(self.favorites.values())

    def get_stats(self) -> Dict:
        if not self.favorites:
            return {'total': 0}
        favs = list(self.favorites.values())
        return {
            'total': len(favs),
            'by_category': dict(Counter(f.category for f in favs)),
            'total_views': sum(f.view_count for f in favs)
        }

print('✅ FavoritesManager defined')

✅ FavoritesManager defined


---

## PART 7: Unified Manager

In [10]:
class UserManager:
    """Unified user management"""

    def __init__(self, storage_dir: Path):
        self.storage_dir = Path(storage_dir)
        self.profiles = UserProfileManager(storage_dir)
        self._history: Dict[str, SearchHistoryManager] = {}
        self._favorites: Dict[str, FavoritesManager] = {}

    def get_history(self, user_id: str) -> SearchHistoryManager:
        if user_id not in self._history:
            self._history[user_id] = SearchHistoryManager(user_id, self.storage_dir)
        return self._history[user_id]

    def get_favorites(self, user_id: str) -> FavoritesManager:
        if user_id not in self._favorites:
            self._favorites[user_id] = FavoritesManager(user_id, self.storage_dir)
        return self._favorites[user_id]

    def sync_profile_counts(self, user_id: str):
        """Sync profile counts with actual data"""
        user = self.profiles.get_user(user_id)
        if user:
            user.total_searches = len(self.get_history(user_id).history)
            user.total_favorites = len(self.get_favorites(user_id).favorites)
            user.update_activity()
            self.profiles._save(user)

    def get_analytics(self, user_id: str) -> Dict:
        user = self.profiles.get_user(user_id)
        if not user:
            return {'error': 'User not found'}
        return {
            'user_id': user_id,
            'name': user.name,
            'email': user.email,
            'searches': self.get_history(user_id).get_patterns(),
            'favorites': self.get_favorites(user_id).get_stats(),
            'preferences': user.preferences.to_dict()
        }

print('✅ UserManager defined')

✅ UserManager defined


---

## PART 8: Initialize System

In [11]:
# Initialize
STORAGE_DIR = Path('v2.4-complete/data/users')
user_manager = UserManager(STORAGE_DIR)

print('✅ System initialized')
print(f'📁 Storage: {STORAGE_DIR}')

✅ System initialized
📁 Storage: v2.4-complete/data/users


---

## PART 9: Create Users

In [12]:
# Create 3 test users
users = [
    ('U001', 'Alice Johnson', 'alice@example.com'),
    ('U002', 'Bob Smith', 'bob@example.com'),
    ('U003', 'Carol Williams', 'carol@example.com'),
]

print('👥 Creating users...')
for user_id, name, email in users:
    user = user_manager.profiles.create_user(user_id, name, email)
    print(f"✅ {name} ({user_id})")

print(f"\n✅ Created {len(users)} users")

👥 Creating users...
✅ Alice Johnson (U001)
✅ Bob Smith (U002)
✅ Carol Williams (U003)

✅ Created 3 users


---

## PART 10: Set Preferences

In [13]:
# Alice - Casual style
user_manager.profiles.update_preferences(
    'U001',
    style=['casual', 'sporty'],
    size='S',
    colors=['blue', 'white', 'gray']
)
print('✅ Alice: Casual/Sporty')

# Bob - Formal style
user_manager.profiles.update_preferences(
    'U002',
    style=['formal', 'elegant'],
    size='L',
    colors=['black', 'navy', 'gray']
)
print('✅ Bob: Formal/Elegant')

# Carol - Vintage style
user_manager.profiles.update_preferences(
    'U003',
    style=['vintage'],
    size='M',
    colors=['red', 'orange', 'yellow']
)
print('✅ Carol: Vintage')

✅ Alice: Casual/Sporty
✅ Bob: Formal/Elegant
✅ Carol: Vintage


---

## PART 11: Add Search History

In [14]:
# Alice searches
alice_history = user_manager.get_history('U001')
alice_history.add_search("blue dress", 15, "P001", 0.23)
alice_history.add_search("white sneakers", 23, "P045", 0.19)
alice_history.add_search("gray jacket", 18, "P123", 0.20)
print('✅ Alice: 3 searches')

# Bob searches
bob_history = user_manager.get_history('U002')
bob_history.add_search("black suit", 12, "P567", 0.18)
bob_history.add_search("navy shirt", 20, "P234", 0.16)
print('✅ Bob: 2 searches')

# Carol searches
carol_history = user_manager.get_history('U003')
carol_history.add_search("vintage dress", 8, "P789", 0.15)
carol_history.add_search("retro accessories", 14, "P890", 0.17)
print('✅ Carol: 2 searches')

✅ Alice: 3 searches
✅ Bob: 2 searches
✅ Carol: 2 searches


---

## PART 12: Add Favorites

In [15]:
# Alice favorites
alice_favs = user_manager.get_favorites('U001')
alice_favs.add('P001', 'Blue Summer Dress', 'dress')
alice_favs.add('P045', 'White Sneakers', 'shoes')
alice_favs.add('P123', 'Gray Jacket', 'jacket')
print('✅ Alice: 3 favorites')

# Bob favorites
bob_favs = user_manager.get_favorites('U002')
bob_favs.add('P567', 'Black Suit', 'suit')
bob_favs.add('P234', 'Navy Shirt', 'shirt')
print('✅ Bob: 2 favorites')

# Carol favorites
carol_favs = user_manager.get_favorites('U003')
carol_favs.add('P789', 'Vintage Dress', 'dress')
carol_favs.add('P890', 'Retro Earrings', 'accessories')
print('✅ Carol: 2 favorites')

✅ Alice: 3 favorites
✅ Bob: 2 favorites
✅ Carol: 2 favorites


---

## PART 13: Record Views

In [16]:
# Alice views favorites
alice_favs.record_view('P001')
alice_favs.record_view('P001')
alice_favs.record_view('P045')
print('✅ Alice: 3 views')

# Bob views
bob_favs.record_view('P567')
print('✅ Bob: 1 view')

# Carol views
carol_favs.record_view('P789')
carol_favs.record_view('P890')
print('✅ Carol: 2 views')

✅ Alice: 3 views
✅ Bob: 1 view
✅ Carol: 2 views


---

## PART 14: Sync Profile Counts

In [17]:
# Sync all user profile counts
print('🔄 Syncing profile counts...')
for user_id in user_manager.profiles.list_users():
    user_manager.sync_profile_counts(user_id)
    user = user_manager.profiles.get_user(user_id)
    print(f"✅ {user.name}: {user.total_searches} searches, {user.total_favorites} favorites")

print('\n✅ Profile counts synchronized')

🔄 Syncing profile counts...
✅ Alice Johnson: 3 searches, 3 favorites
✅ Bob Smith: 2 searches, 2 favorites
✅ Carol Williams: 2 searches, 2 favorites

✅ Profile counts synchronized


---

## PART 15: View Analytics

In [18]:
print('\n📊 USER ANALYTICS')
print('='*60)

for user_id in ['U001', 'U002', 'U003']:
    analytics = user_manager.get_analytics(user_id)
    print(f"\n{analytics['name']} ({user_id}):")
    print(json.dumps(analytics, indent=2))


📊 USER ANALYTICS

Alice Johnson (U001):
{
  "user_id": "U001",
  "name": "Alice Johnson",
  "email": "alice@example.com",
  "searches": {
    "total": 3,
    "unique": 3,
    "top_terms": {
      "blue": 1,
      "dress": 1,
      "white": 1,
      "sneakers": 1,
      "gray": 1
    },
    "avg_response": 0.20666666666666667
  },
  "favorites": {
    "total": 3,
    "by_category": {
      "dress": 1,
      "shoes": 1,
      "jacket": 1
    },
    "total_views": 3
  },
  "preferences": {
    "style": [
      "casual",
      "sporty"
    ],
    "size": "S",
    "colors": [
      "blue",
      "white",
      "gray"
    ],
    "categories": []
  }
}

Bob Smith (U002):
{
  "user_id": "U002",
  "name": "Bob Smith",
  "email": "bob@example.com",
  "searches": {
    "total": 2,
    "unique": 2,
    "top_terms": {
      "black": 1,
      "suit": 1,
      "navy": 1,
      "shirt": 1
    },
    "avg_response": 0.16999999999999998
  },
  "favorites": {
    "total": 2,
    "by_category": {
      "

---

## PART 16: System Statistics

In [19]:
print('\n📊 SYSTEM STATISTICS')
print('='*60)

stats = user_manager.profiles.get_stats()
print(json.dumps(stats, indent=2))

print('\n📁 Storage Files:')
for f in sorted(STORAGE_DIR.glob('*.json')):
    size = f.stat().st_size
    print(f"  {f.name:<25} {size:>6} bytes")


📊 SYSTEM STATISTICS
{
  "total_users": 3,
  "avg_searches": 2.3333333333333335,
  "avg_favorites": 2.3333333333333335
}

📁 Storage Files:
  favorites_U001.json          642 bytes
  favorites_U002.json          411 bytes
  favorites_U003.json          449 bytes
  history_U001.json            495 bytes
  history_U002.json            328 bytes
  history_U003.json            337 bytes
  users.json                  1351 bytes


---

##  PART 17: Summary

In [20]:
print('\n' + '='*60)
print('🎉 v2.4 USER MANAGEMENT COMPLETE!')
print('='*60)

print('\n✅ Completed:')
print('  • User profiles (3 users)')
print('  • Search history (7 searches)')
print('  • Favorites (7 items)')
print('  • Analytics & statistics')

print('\n📊 System:')
print(f"  • Users: {len(user_manager.profiles.list_users())}")
print(f"  • Storage: {STORAGE_DIR}")
print(f"  • Files: {len(list(STORAGE_DIR.glob('*.json')))} JSON files")

print('\n🎯 Next: Personalization Engine')
print('='*60)


🎉 v2.4 USER MANAGEMENT COMPLETE!

✅ Completed:
  • User profiles (3 users)
  • Search history (7 searches)
  • Favorites (7 items)
  • Analytics & statistics

📊 System:
  • Users: 3
  • Storage: v2.4-complete/data/users
  • Files: 7 JSON files

🎯 Next: Day 2 - Personalization Engine
